liberias 

In [1]:
import requests
import pandas as pd
import os
import json
from pandas import json_normalize

Configuración inicial

In [8]:
BASE_URL = "https://api.spaceflightnewsapi.net/v4/articles/"
LIMIT = 500
STATE_FILE = "state.json"
MAX_ARTICLES = 1000

Cargar estado desde archivo (offset)

In [9]:
def load_state():
    if os.path.exists(STATE_FILE):
        with open(STATE_FILE, "r") as f:
            return json.load(f)
    return {
        "offset": 0
    }

state = load_state()
current_offset = state["offset"]
print(f"\n Cargando artículos desde offset: {current_offset}")


 Cargando artículos desde offset: 0


Obtener la data de acuerdo a los parametros que se requieren y el ultimo offset si es que se ejecuta nuevamente el workflow

In [ ]:
def fetch_articles(offset):
    params = {
        "limit": LIMIT,
        "offset": offset,
        "ordering": "updated_at"
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        raise Exception(f"Error {response.status_code}: {response.text}")
    return response.json()["results"]

data = []
if current_offset == 0:
    stop = False
    while stop == False:
        data_i = fetch_articles(current_offset)
        data.extend(data_i)
        if len(data_i) < LIMIT:
            stop = True
        current_offset += LIMIT
        print(f" Cargando artículos desde offset: {current_offset}")
    if not data:
        print(" No hay más artículos disponibles.")
        exit()

data



: 

In [6]:
len(data)

28677

1000 datos

In [ ]:
def fetch_articles(offset):
    params = {
        "limit": LIMIT,
        "offset": offset,
        "ordering": "updated_at"
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        raise Exception(f"Error {response.status_code}: {response.text}")
    return response.json()["results"]

data = []
stop = False

while not stop:
    data_i = fetch_articles(current_offset)
    if not data_i:
        stop = True
        break
    
    data.extend(data_i)
    current_offset += LIMIT

    print(f" Cargando artículos desde offset: {current_offset}")
    
    # Detener si ya se alcanzaron 1000 registros
    if len(data) >= MAX_ARTICLES:
        stop = True

#  Cortar a exactamente 1000 si por alguna razón se pasaron
data = data[:MAX_ARTICLES]

print(f"Total de artículos cargados: {len(data)}")

 Cargando artículos desde offset: 500
 Cargando artículos desde offset: 1000
Total de artículos cargados: 1000


In [11]:
mil_df = json_normalize(data)

In [12]:
mil_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1000 non-null   int64 
 1   title         1000 non-null   object
 2   authors       1000 non-null   object
 3   url           1000 non-null   object
 4   image_url     1000 non-null   object
 5   news_site     1000 non-null   object
 6   summary       1000 non-null   object
 7   published_at  1000 non-null   object
 8   updated_at    1000 non-null   object
 9   featured      1000 non-null   bool  
 10  launches      1000 non-null   object
 11  events        1000 non-null   object
dtypes: bool(1), int64(1), object(10)
memory usage: 87.0+ KB


Convertir de json a dataframe

In [13]:
raw_df = json_normalize(data)

ver data RAW

In [ ]:
raw_df.head()

Descripción de la data

In [ ]:
raw_df.describe(include='all')

Ver toda la información de la data para poder identificar la data que tenemos, de que tipo  y el conteo del numero de filas obtenidas

In [ ]:
raw_df.info()

Nombre de Columnas 

In [17]:
print("Columnas RAW:")
print(raw_df.columns.tolist())

Columnas RAW:
['id', 'title', 'authors', 'url', 'image_url', 'news_site', 'summary', 'published_at', 'updated_at', 'featured', 'launches', 'events']


Crear la carpeta 'raw' si no existe

In [19]:
os.makedirs("raw", exist_ok=True)

Almacenamiento local de la primera extración raw

In [20]:
datetime = pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M-%S')
output_file = f"raw/{datetime}_articulos_espaciales_test.csv"
raw_df.to_csv(output_file, sep=';', index=False)
print(f"\nArchivo RAW guardado en: {output_file}")


Archivo RAW guardado en: raw/2025-07-13_15-53-54_articulos_espaciales_test.csv


Leer el archivo

In [ ]:
output_file = f"raw/{datetime}_articulos_espaciales_test.csv"
df = pd.read_csv(output_file, sep=';')

df

LIMPIEZA

Verificar si hay duplicados, si es FALSE es porque no hay 

In [22]:
df.duplicated(subset=['id']).all()

False

Borrar duplicados

In [ ]:
df.drop_duplicates(subset=['id'])

Organizar la data para que aparezca del primer dato insertado en adelante

In [ ]:
df.sort_values(by='updated_at', ascending=True)

Formateando y normalizando la data y creación de dos columnas las cuales guardan la data con la hora exacta para no perder data

In [ ]:
df['published_datetime'] = pd.to_datetime(df['published_at'])
df['published_at'] = pd.to_datetime(df['published_at']).dt.date
df['updated_datetime'] = pd.to_datetime(df['updated_at'])
df['updated_at'] = pd.to_datetime(df['updated_at']).dt.date
df['id'] = df['id'].astype(int)
#df['featured'] = df['featured'].astype(bool) ya viene booleano

# Limpiar columnas de texto
text_columns = ['title', 'authors', 'url', 'image_url', 'news_site', 'summary']
for col in text_columns:
    df[col] = df[col].astype(str).str.strip()

verificar cambios

In [ ]:
df.info()


Crear la carpeta de Staging si no existe

In [28]:
os.makedirs("staging", exist_ok=True)

Guardar la data en el cvs de staging

In [29]:
datetime = pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M-%S')
output_file = f"staging/{datetime}_articulos_espaciales_test.csv"
df.to_csv(output_file, sep=';', index=False)
print(f"Archivo staging guardado en: {output_file}")

Archivo staging guardado en: staging/2025-07-13_15-54-20_articulos_espaciales_test.csv


Actualizar el JSON con el ultimo offset agregado 

In [ ]:
def save_state(offset):
    with open(STATE_FILE, "w") as f:
        json.dump({"offset": offset}, f)

new_offset = current_offset + LIMIT
save_state(new_offset)
print(f"Nuevo offset guardado: {new_offset}")
